In [86]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

def get_corner_stats(team_url:str,is_home:bool):
    html = requests.get(team_url).content

    soup = BeautifulSoup(html, 'html.parser')

    team_name = soup.find("meta", {"name":"description"})["content"]
    team_name = team_name.split(",")[0].replace('Team: ','')
    first_half_corners_for = []
    second_half_corners_for = []
    match_corners_for = []
    first_half_corners_against = []
    second_half_corners_against = []
    match_corners_against = []
    total_match_corners = []
    total_first_half_corners = []
    total_second_half_corners =[]
    home_or_away = []
    corners_balance =[]



    table_rows = soup.findAll("tr")
    for rows in table_rows:
        if rows.find("span", class_="span_half_corner hide"):
            continue
        match = rows.find("td", class_="match_home")
        if match != None:
            home_team = match.find('a').text
        else:
            continue
        match = rows.find("td", class_="match_away")
        if match != None:
            away_team = match.find('a').text
        half_corners = rows.find("span", class_="span_half_corner").text
        match_corners = rows.find("span", class_="span_match_corner").text
        home_half_corners = half_corners.replace('(','').replace(')','').split('-')[0]
        away_half_corners = half_corners.replace('(','').replace(')','').split('-')[1]
        home_match_corners = match_corners.split(' ')[0]
        away_match_corners = match_corners.split(' ')[-1]
        home_second_half_corners = int(home_match_corners) - int(home_half_corners)
        away_second_half_corners = int(away_match_corners) - int(away_half_corners)
        if home_team == team_name:
            balance = int(home_match_corners) - int(away_match_corners)
            first_half_corners_for.append(int(home_half_corners))
            second_half_corners_for.append(int(home_second_half_corners))
            match_corners_for.append(int(home_match_corners))
            first_half_corners_against.append(int(away_half_corners))
            second_half_corners_against.append(int(away_second_half_corners))
            match_corners_against.append(int(away_match_corners))
            total_match_corners.append(int(home_match_corners) + int(away_match_corners))
            total_first_half_corners.append(int(home_half_corners) + int(away_half_corners))
            total_second_half_corners.append(int(home_second_half_corners) + int(away_second_half_corners))
            home_or_away.append('Home')
            corners_balance.append(balance)

        else:
            balance = int(away_match_corners) - int(home_match_corners)
            first_half_corners_for.append(int(away_half_corners))
            second_half_corners_for.append(int(away_second_half_corners))
            match_corners_for.append(int(away_match_corners))
            first_half_corners_against.append(int(home_half_corners))
            second_half_corners_against.append(int(home_second_half_corners))
            match_corners_against.append(int(home_match_corners))
            total_match_corners.append(int(home_match_corners) + int(away_match_corners))
            total_first_half_corners.append(int(home_half_corners) + int(away_half_corners))
            total_second_half_corners.append(int(home_second_half_corners) + int(away_second_half_corners))
            home_or_away.append('Away')
            corners_balance.append(balance)

    df = pd.DataFrame(list(zip(
            first_half_corners_for,
            second_half_corners_for,
            match_corners_for,
            first_half_corners_against,
            second_half_corners_against,
            match_corners_against,
            total_match_corners,
            total_first_half_corners,
            total_second_half_corners,
            home_or_away,
            corners_balance,
        )),
        columns=[
            'first_half_corners_for',
            'second_half_corners_for',
            'match_corners_for',
            'first_half_corners_against',
            'second_half_corners_against',
            'match_corners_against',
            'total_match_corners',
            'total_first_half_corners',
            'total_second_half_corners',
            'home_or_away',
            'corners_balance',

        ])
    if is_home:
        return (team_name, df[(df.home_or_away == "Home")])
    else:
        return (team_name, df[(df.home_or_away == "Away")])


#team_url = input("Url totalcorner do time da casa\n")
#team2_url = input("Url totalcorner do time visitante\n")
team_url = "https://www.totalcorner.com/team/view/7"
team2_url = "https://www.totalcorner.com/team/view/4"

 
home_name, df_home = get_corner_stats(team_url,True)
away_name, df_away = get_corner_stats(team2_url,False)
print('Jogando em casa', home_name)
print('FT 80% das partidas - de ', df_home.total_match_corners.quantile(0.8), "escanteios")
print('FT 90% das partidas + de ', df_home.total_match_corners.quantile(0.1), "escanteios")
print('1H 80% das partidas - ', df_home.total_first_half_corners.quantile(0.8), "escanteios")
print('1H 80% das partidas + de ', df_home.total_first_half_corners.quantile(0.2), "escanteios")
print('1H partidas com menos de 6 ', round((df_home.total_first_half_corners < 6).mean()*100), "%")
print('Partidas com saldo de escanteios positivo: ', round((df_home.corners_balance > 0).mean()*100), "%")
print('Partidas com saldo de escanteios negativo: ', round((df_home.corners_balance < 0).mean()*100), "%")
print('Partidas escanteios empatados: ', round((df_home.corners_balance == 0).mean()*100), "%")
display(df_home)
print('Jogando fora de casa', away_name)
print('FT 80% das partidas - de ', df_away.total_match_corners.quantile(0.8), "escanteios")
print('FT 90% das partidas + de ', df_away.total_match_corners.quantile(0.1), "escanteios")
print('1H 80% das partidas - ', df_away.total_first_half_corners.quantile(0.8), "escanteios")
print('1H 80% das partidas + de ', df_away.total_first_half_corners.quantile(0.2), "escanteios")
print('1H partidas com menos de 6 ', round((df_away.total_first_half_corners < 6).mean()*100), "%")
print('Partidas com saldo de escanteios positivo: ', round((df_away.corners_balance > 0).mean()*100), "%")
print('Partidas com saldo de escanteios negativo: ', round((df_away.corners_balance < 0).mean()*100), "%")
print('Partidas escanteios empatados: ', round((df_away.corners_balance == 0).mean()*100), "%")
display(df_away)


Jogando em casa Newcastle
FT 80% das partidas - de  13.200000000000001 escanteios
FT 90% das partidas + de  6.0 escanteios
1H 80% das partidas -  7.0 escanteios
1H 80% das partidas + de  3.8000000000000003 escanteios
1H partidas com menos de 6  53 %
Partidas com saldo de escanteios positivo:  87 %
Partidas com saldo de escanteios negativo:  13 %
Partidas escanteios empatados:  0 %


,first_half_corners_for,second_half_corners_for,match_corners_for,first_half_corners_against,second_half_corners_against,match_corners_against,total_match_corners,total_first_half_corners,total_second_half_corners,home_or_away,corners_balance
1,6,4,10,1,2,3,13,7,6,Home,7
4,5,1,6,2,5,7,13,7,6,Home,-1
6,0,4,4,1,1,2,6,1,5,Home,2
7,4,1,5,0,1,1,6,4,2,Home,4
10,7,3,10,2,3,5,15,9,6,Home,5
11,5,7,12,2,2,4,16,7,9,Home,8
14,4,5,9,3,0,3,12,7,5,Home,6
16,2,4,6,1,1,2,8,3,5,Home,4
17,3,2,5,3,0,3,8,6,2,Home,2
18,3,2,5,1,2,3,8,4,4,Home,2


Jogando fora de casa Man Utd
FT 80% das partidas - de  12.600000000000001 escanteios
FT 90% das partidas + de  6.7 escanteios
1H 80% das partidas -  5.6000000000000005 escanteios
1H 80% das partidas + de  2.8000000000000003 escanteios
1H partidas com menos de 6  75 %
Partidas com saldo de escanteios positivo:  0 %
Partidas com saldo de escanteios negativo:  88 %
Partidas escanteios empatados:  12 %


,first_half_corners_for,second_half_corners_for,match_corners_for,first_half_corners_against,second_half_corners_against,match_corners_against,total_match_corners,total_first_half_corners,total_second_half_corners,home_or_away,corners_balance
1,2,3,5,2,4,6,11,4,7,Away,-1
3,2,1,3,2,2,4,7,4,3,Away,-1
9,3,2,5,3,5,8,13,6,7,Away,-3
10,2,1,3,0,7,7,10,2,8,Away,-4
15,1,4,5,3,3,6,11,4,7,Away,-1
16,1,3,4,6,6,12,16,7,9,Away,-8
17,1,2,3,1,2,3,6,2,4,Away,0
22,3,2,5,2,5,7,12,5,7,Away,-2
